In [2]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [3]:
import pandas as pd
mnist.target = pd.get_dummies(mnist.target).astype(int).values
mnist.data = mnist.data / 255.0

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.4, random_state=0)

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input

# pretrain the first layer using an autoencoder
encoding_dim = 32
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input=input_img, output=decoded)
encoder = Model(input=input_img, output=encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))
autoencoder.compile(optimizer='sgd', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train,
                nb_epoch=1,
                batch_size=256,
                shuffle=True)

classifier = Dense(10, activation='softmax')(encoded)
model = Model(input=input_img, output=classifier)

from keras import optimizers
optimizer = optimizers.SGD(lr=0.01, momentum=0.5, decay=0.0, nesterov=False)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/1
42000/42000 [==============================] - 11s - loss: 0.6936    
Train on 42000 samples, validate on 28000 samples
Epoch 1/10
42000/42000 [==============================] - 2s - loss: 0.4906 - acc: 0.8641 - val_loss: 0.3086 - val_acc: 0.9103
Epoch 2/10
42000/42000 [==============================] - 2s - loss: 0.2710 - acc: 0.9222 - val_loss: 0.2538 - val_acc: 0.9264
Epoch 3/10
42000/42000 [==============================] - 2s - loss: 0.2267 - acc: 0.9355 - val_loss: 0.2287 - val_acc: 0.9342
Epoch 4/10
42000/42000 [==============================] - 2s - loss: 0.2003 - acc: 0.9444 - val_loss: 0.2070 - val_acc: 0.9406
Epoch 5/10
42000/42000 [==============================] - 2s - loss: 0.1811 - acc: 0.9491 - val_loss: 0.1968 - val_acc: 0.9427
Epoch 6/10
42000/42000 [==============================] - 2s - loss: 0.1650 - acc: 0.9534 - val_loss: 0.1929 - val_acc: 0.9441
Epoch 7/10
42000/42000 [==============================] - 2s - loss: 0.1531 - acc: 0.9557 - val_loss: 0.1779